# Create Submission File

## Imports and get data

In [34]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from HelpFunctions.date_and_time import most_recent_thursday, split_time

from DAX.HelpFunctions.get_dax_data import get_dax_data
from DAX.Models import baseline as dax_bl
from DAX.Models import garch11 as dax_garch

from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from Energy.Models import baseline as en_bl
from Energy.Models import Model4_population as en_m4_pop
from datetime import datetime
import pytz

# import importlib
# importlib.reload(Energy.Models)

# import importlib
# importlib.reload(Energy.Models)

### DAX: Get and prepare Data
Use tuesday as last datapoint

In [114]:
# Insert Date of Wednesday (forecast-day) here. The current day has to be wednesday!!
start_date = datetime(2024, 1, 17, 18, 0, 0)
assert start_date.weekday() == 2

In [115]:
df_dax = get_dax_data()
# pretend_start_date = date(2023, 11, 22, 11, 00, 0, tzinfo=pytz.timezone('Europe/Berlin'))

df_dax = df_dax[df_dax.index.date <= start_date.date()]

In [116]:
df_dax.tail()

,Close,ret1,ret2,ret3,ret4,ret5
Date,,,,,,
2024-01-11 00:00:00+01:00,16547.029297,-0.859180,-0.850485,-1.018792,-0.284731,-0.423708
2024-01-12 00:00:00+01:00,16704.560547,0.947518,0.088338,0.097034,-0.071273,0.662787
2024-01-15 00:00:00+01:00,16622.220703,-0.494137,0.453381,-0.405799,-0.397104,-0.565411
2024-01-16 00:00:00+01:00,16571.679688,-0.304520,-0.798657,0.148861,-0.710319,-0.701624
2024-01-17 00:00:00+01:00,16431.689453,-0.848344,-1.152864,-1.647001,-0.699483,-1.558663


### Energy: Get and prepare Data

In [122]:
from Energy.HelpFunctions.get_energy_data import fetch_energy_data
fetch_energy_data()

  0%|          | 0/265 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 265/265 [00:45<00:00,  5.89it/s]


In [123]:
df_en = get_energy_data()
df_en = prepare_data(df_en)

In [126]:
df_en.tail(10)

,gesamt
date_time,
2024-01-17 09:00:00,73.11700
2024-01-17 10:00:00,73.06200
2024-01-17 11:00:00,72.65925
2024-01-17 12:00:00,72.72250
2024-01-17 13:00:00,71.57850
2024-01-17 14:00:00,70.56925
2024-01-17 15:00:00,69.19300
2024-01-17 16:00:00,17.30400
2024-01-17 17:00:00,70.99425


In [119]:
df_en = df_en.loc[df_en.index <= start_date]

In [1]:
# Check if still required next week ...
df_en.loc['2024-01-17 16:00:00', 'gesamt'] = (df_en.loc['2024-01-17 15:00:00', 'gesamt'] +df_en.loc['2024-01-17 17:00:00', 'gesamt'])/2
df_en.tail()

NameError: name 'df_en' is not defined

## Run models for the data

### DAX

In [106]:
from DAX.Models import arma_garch_11 as ag
from DAX.Models import baseline as bl

import importlib
importlib.reload(ag)
importlib.reload(bl)


# dax_bl.baseline(df_dax)
# forecast_dax = dax_garch.garch11(df_dax)
forecast_dax = mix_models([ag.garch11_norm_1y, bl.baseline_1y], [1,1], df_dax, 'DAX')
forecast_dax


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-18,DAX,1 day,-1.558635,-0.423302,0.018505,0.505234,1.547897
1,2024-01-19,DAX,2 day,-2.145728,-0.653572,0.010310,0.720111,2.215027
2,2024-01-22,DAX,5 day,-2.842990,-0.841287,0.070235,0.986638,2.776736
3,2024-01-23,DAX,6 day,-2.995225,-0.869402,0.100907,1.047762,3.040285
4,2024-01-24,DAX,7 day,-3.287998,-0.926635,0.103120,1.232834,3.306357


### Energy  

In [107]:
from Energy.Models import mstl
from Energy.Models.baseline import baseline
from HelpFunctions.mix_models import mix_models_per_horizon
from Energy.Models.Model4_sunhours import model4_sunhours
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_population import model4_population
from Energy.Models import Model5

# en_bl.baseline(df_en, LAST_IDX=-1, offset_horizons=11)

import importlib
importlib.reload(Model5)
importlib.reload(mstl)

weights_m5_bl_m4_x = [1,1]
functions_m5_bl_m4_x = [mstl.mstl, Model5.model5]




# forecast_en = en_m4_pop.model4_population(df_en, offset_horizons=15)
forecast_en = mix_models(functions_m5_bl_m4_x, weights_m5_bl_m4_x, df_en, target="energy", offset_horizons=6)
forecast_en

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-19 12:00:00,energy,36 hour,58.324867,64.902470,68.517759,70.959155,75.874334
1,2024-01-19 16:00:00,energy,40 hour,56.282704,62.124835,65.431514,68.004441,72.945365
2,2024-01-19 20:00:00,energy,44 hour,52.989286,58.142251,60.948244,63.430792,68.432895
3,2024-01-20 12:00:00,energy,60 hour,50.255321,56.832334,59.979786,62.221662,66.729531
4,2024-01-20 16:00:00,energy,64 hour,49.067564,54.909106,57.747947,60.121355,64.654968
5,2024-01-20 20:00:00,energy,68 hour,46.397904,51.550280,53.888435,56.171464,60.766256


### Infections

In [108]:
forecast_infections = {
    'forecast_date': [start_date.strftime('%Y-%m-%d')] * 5,
    'target': ['infections'] * 5,
    'horizon': ['0 week', '1 week', '2 week', '3 week', '4 week'],
    # 'q0.025': ['NA'] * 5,
    # 'q0.25': ['NA'] * 5,
    # 'q0.5': ['NA'] * 5,
    # 'q0.75': ['NA'] * 5,
    # 'q0.975': ['NA'] * 5,
    'q0.025': [np.nan] * 5,
    'q0.25': [np.nan] * 5,
    'q0.5': [np.nan] * 5,
    'q0.75': [np.nan] * 5,
    'q0.975': [np.nan] * 5
}

forecast_infections = pd.DataFrame(forecast_infections)

In [109]:
forecast_infections

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,infections,0 week,NaN,NaN,NaN,NaN,NaN
1,2024-01-17,infections,1 week,NaN,NaN,NaN,NaN,NaN
2,2024-01-17,infections,2 week,NaN,NaN,NaN,NaN,NaN
3,2024-01-17,infections,3 week,NaN,NaN,NaN,NaN,NaN
4,2024-01-17,infections,4 week,NaN,NaN,NaN,NaN,NaN


## Merge results

Check results: Are they valid, forecast_date is right? The last entry might be cut off

In [110]:
forecasts = pd.concat([forecast_dax,forecast_en, forecast_infections]).reset_index().drop(columns=['index'])
forecasts

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-18,DAX,1 day,-1.558635,-0.423302,0.018505,0.505234,1.547897
1,2024-01-19,DAX,2 day,-2.145728,-0.653572,0.010310,0.720111,2.215027
2,2024-01-22,DAX,5 day,-2.842990,-0.841287,0.070235,0.986638,2.776736
3,2024-01-23,DAX,6 day,-2.995225,-0.869402,0.100907,1.047762,3.040285
4,2024-01-24,DAX,7 day,-3.287998,-0.926635,0.103120,1.232834,3.306357
5,2024-01-19 12:00:00,energy,36 hour,58.324867,64.902470,68.517759,70.959155,75.874334
6,2024-01-19 16:00:00,energy,40 hour,56.282704,62.124835,65.431514,68.004441,72.945365
7,2024-01-19 20:00:00,energy,44 hour,52.989286,58.142251,60.948244,63.430792,68.432895
8,2024-01-20 12:00:00,energy,60 hour,50.255321,56.832334,59.979786,62.221662,66.729531
9,2024-01-20 16:00:00,energy,64 hour,49.067564,54.909106,57.747947,60.121355,64.654968


If everything seems good: Override forecast_date

In [111]:
forecasts['forecast_date'] = [start_date.strftime('%Y-%m-%d') for _ in range(len(forecasts))]

In [112]:
forecasts

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,DAX,1 day,-1.558635,-0.423302,0.018505,0.505234,1.547897
1,2024-01-17,DAX,2 day,-2.145728,-0.653572,0.010310,0.720111,2.215027
2,2024-01-17,DAX,5 day,-2.842990,-0.841287,0.070235,0.986638,2.776736
3,2024-01-17,DAX,6 day,-2.995225,-0.869402,0.100907,1.047762,3.040285
4,2024-01-17,DAX,7 day,-3.287998,-0.926635,0.103120,1.232834,3.306357
5,2024-01-17,energy,36 hour,58.324867,64.902470,68.517759,70.959155,75.874334
6,2024-01-17,energy,40 hour,56.282704,62.124835,65.431514,68.004441,72.945365
7,2024-01-17,energy,44 hour,52.989286,58.142251,60.948244,63.430792,68.432895
8,2024-01-17,energy,60 hour,50.255321,56.832334,59.979786,62.221662,66.729531
9,2024-01-17,energy,64 hour,49.067564,54.909106,57.747947,60.121355,64.654968


### Check df & Save csv-File

In [113]:
from HelpFunctions.check_submission import check_submission_df

check_submission_df(forecasts)
forecasts.to_csv(f'./Abgaben/{start_date.strftime("%Y%m%d")}_JakePeralta.csv', na_rep='NA', index=False)

Start checking...
---------------------------
Checking the Columns...
Checking type of columns...
----WARNING: Some elements in q0.025 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.25 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.5 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.75 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.975 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

Checking if the Dates make sense...
Checking targets...
Checking quantiles...
Assume that -- infections -- is your NaN-target. Please DOUBLECHECK if this is correct.
---------------------------
Looks good!


/Users/elias/Desktop/PTSFC/HelpFunctions/check_submission.py:175: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  diffs[0] = 0
/Users/elias/Desktop/PTSFC/HelpFunctions/check_submission.py:175: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  diffs[0] = 0
/Users/elias/Desktop/PTSFC/HelpFunctions/check_submission.py:175: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  diffs[0] = 0
/Users/elias/Desktop/PTSFC/HelpFunctions/check